Необхідні для роботи імпорти

In [1]:
import re, chardet, math
from collections import Counter
import pandas as pd

Змінні, які необхідні для роботи коду

In [2]:
file_path = r"C:\Users\blood\OneDrive\Рабочий стол\koran.txt"

Відкриваємо та форматуємо текст, щоб нам було зручно з ним працювати

In [3]:
with open(file_path, "rb") as f:
    raw_data = f.read()
koduvannya = chardet.detect(raw_data)["encoding"]
print(f"Кодування: {koduvannya}")

def obrobka_text(filename, remove_spaces=False):
    with open(filename, "r", encoding=koduvannya, errors="replace") as file:
        text = file.read().lower()
    text = text.replace("ё", "е").replace("ъ", "ь")
    text = re.sub(r'[^а-я ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if remove_spaces:
        text = text.replace(" ", "")
    return text

koranchik_z_probilamy = obrobka_text(file_path, remove_spaces=False)
koranchik_bez_probiliv = obrobka_text(file_path, remove_spaces=True)
print(f"Довжина з пробілами: {len(koranchik_z_probilamy)}, без пробілів: {len(koranchik_bez_probiliv)}")

Кодування: utf-8
Довжина з пробілами: 1665627, без пробілів: 1391512


In [4]:
def chastoty_bukv(tekst):
    N = len(tekst)
    counts = Counter(tekst)
    freq = {sym: count / N for sym, count in counts.items()}
    return freq, counts, N

def chastoty_bigram(tekst):
    bigrams = [tekst[i:i+2] for i in range(len(tekst) - 1)]
    N = len(bigrams)
    counts = Counter(bigrams)
    freq = {bg: count / N for bg, count in counts.items()}
    return freq, counts, N

def entropiya(freq_dict, n=1):
    """
    прийматиме freq_dict - словник {символ: ймовірність}
    n - довжина грам (1 для букв, 2 для біграм)
    """
    return -sum(p * math.log2(p) for p in freq_dict.values() if p > 0) / n

Обчислення H1 та H2 з пробілами

In [ ]:
freq_letters, counts_letters, N_letters = chastoty_bukv(koranchik_z_probilamy)
freq_bigrams, counts_bigrams, N_bigrams = chastoty_bigram(koranchik_z_probilamy)

H1_with_spaces = entropiya(freq_letters, n=1)
H2_with_spaces = entropiya(freq_bigrams, n=2)

print(f"H1 (з пробілами): {H1_with_spaces:.6f}")
print(f"H2(з пробілами): {H2_with_spaces:.6f}")

H1: 4.346580
H2: 3.898952


Обчислення H1 та H2 без пробілів

In [ ]:
freq_letters_ns, counts_letters_ns, N_letters_ns = chastoty_bukv(tekst_bez_probiliv)
freq_bigrams_ns, counts_bigrams_ns, N_bigrams_ns = chastoty_bigram(tekst_bez_probiliv)

H1_no_spaces = entropiya(freq_letters_ns, n=1)
H2_no_spaces = entropiya(freq_bigrams_ns, n=2)

print(f"H1 (без пробілів): {H1_with_spaces:.6f}")
print(f"H2 (без пробілів): {H2_with_spaces:.6f}")

NameError: name 'tekst_bez_probiliv' is not defined

Збереження в ексель

In [ ]:

df_letters_spaces = pd.DataFrame(
    [(sym, counts_letters[sym], freq_letters[sym]) for sym in freq_letters],
    columns=["Символ", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_letters_no_spaces = pd.DataFrame(
    [(sym, counts_letters_ns[sym], freq_letters_ns[sym]) for sym in freq_letters_ns],
    columns=["Символ", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_spaces = pd.DataFrame(
    [(bg, counts_bigrams[bg], freq_bigrams[bg]) for bg in freq_bigrams],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_no_spaces = pd.DataFrame(
    [(bg, counts_bigrams_ns[bg], freq_bigrams_ns[bg]) for bg in freq_bigrams_ns],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_letters_spaces.to_excel("letters_with_spaces.xlsx", index=False)
df_letters_no_spaces.to_excel("letters_no_spaces.xlsx", index=False)
df_bigrams_spaces.to_excel("bigrams_with_spaces.xlsx", index=False)
df_bigrams_no_spaces.to_excel("bigrams_no_spaces.xlsx", index=False)

print("Усі таблиці збережено частот та біграм збережено")


Результати збережено у 'chastoty_bukv.xlsx' та 'chastoty_bigram.xlsx'
